# Checkpoint Three: Cleaning Data

Now you are ready to clean your data. Before starting coding, provide the link to your dataset below.

My dataset: https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand

Import the necessary libraries and create your dataframe(s).

In [2]:
import pandas as pd

file_path = 'hotel_bookings.csv'
df = pd.read_csv(file_path)

## Missing Data

Test your dataset for missing data and handle it as needed. Make notes in the form of code comments as to your thought process.

In [3]:
#print(df.isnull().sum())
df['children'] = df['children'].fillna(0) # there are only 4 data points in children with missing data. As 0 is the most common occurrence in this column I am filling with 0.
df['country'].describe() # checking which country code is the most frequent.
df['country'] = df['country'].fillna('PRT') # there are only 488 records showing as "NULL". filling with country code PRT as it occurs most frequently.

# 'agent' and 'company' both have large counts of missing data as most reservations occur from transient customers and not from other business related leads. setting nulls to 0 for consistency.
df['agent'] = df['agent'].fillna(0)
df['company'] = df['company'].fillna(0)

#checking changes were committed. Success = True
print(df.isnull().sum())



hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
children                          0
babies                            0
meal                              0
country                           0
market_segment                    0
distribution_channel              0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
agent                             0
company                           0
days_in_waiting_list              0
customer_type                     0
adr                         

## Irregular Data

Detect outliers in your dataset and handle them as needed. Use code comments to make notes about your thought process.

In [4]:
# a visual inspection from the EDA phase indicated that most of the data was grouped together making it usable. 
# due to this I am only removing the top and bottom 10%
low = df['adr'].quantile(0.10)
up = df['adr'].quantile(0.90)

df_iqr = df[(df['adr'] >= low) & (df['adr'] <= up)] # new df designed to reference the middle 80% of data

df_iqr['adr'].describe() # visual check 

count    95871.000000
mean        97.546409
std         28.205058
min         50.000000
25%         75.000000
50%         94.500000
75%        118.000000
max        164.000000
Name: adr, dtype: float64

In [5]:
df_iqr['lead_time'].describe() # checking stats for outliers
low = df['lead_time'].quantile(0.10) # setting lower 10%
up = df['lead_time'].quantile(0.90) # setting upper 10%

df_iqr = df_iqr[(df_iqr['lead_time'] >= low) & (df_iqr['lead_time'] <= up)] # reference the middle 80% of data

df_iqr['lead_time'].describe() # visual check 

count    77271.000000
mean        90.548964
std         71.045507
min          3.000000
25%         29.000000
50%         73.000000
75%        142.000000
max        265.000000
Name: lead_time, dtype: float64

#### Parsing Dates

The title says it all, we have multiple columns with separate date information. Will combine and remove extra columns in the next section.

In [6]:
# the arrival date month is stored as text and needs to be converted to a number
df_iqr['arrival_date_month'] = pd.to_datetime(df_iqr['arrival_date_month'], format='%B').dt.month
# renaming columns to work with pandas to_datetime
df_iqr.rename(columns={'arrival_date_year': 'year','arrival_date_month': 'month','arrival_date_day_of_month': 'day'}, inplace=True)
df_iqr['check_in_date'] = pd.to_datetime(df_iqr[['month', 'day', 'year']]) #creating new column with newly parsed date
df_iqr.head() #visual check

,hotel,is_canceled,lead_time,year,month,arrival_date_week_number,day,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,check_in_date
2,Resort Hotel,0,7,2015,7,27,1,0,1,1,...,0.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01
3,Resort Hotel,0,13,2015,7,27,1,0,1,1,...,304.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02,2015-07-01
4,Resort Hotel,0,14,2015,7,27,1,0,2,2,...,240.0,0.0,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01
5,Resort Hotel,0,14,2015,7,27,1,0,2,2,...,240.0,0.0,0,Transient,98.0,0,1,Check-Out,2015-07-03,2015-07-01
7,Resort Hotel,0,9,2015,7,27,1,0,2,2,...,303.0,0.0,0,Transient,103.0,0,1,Check-Out,2015-07-03,2015-07-01


## Unnecessary Data

Look for the different types of unnecessary data in your dataset and address it as needed. Make sure to use code comments to illustrate your thought process.

In [7]:
# creating iterable list to drop columns that are irrelevant to the task at hand.
drop_list = [
    'year', # dates already parsed, removing this extra column
    'month', # dates already parsed, removing this extra column
    'arrival_date_week_number', # dates already parsed, removing this extra column
    'day', # dates already parsed, removing this extra column
    'adults', # head count is not relevant to this analysis as we are focusing on rooms reserved instead
    'children',# head count is not relevant to this analysis as we are focusing on rooms reserved instead
    'babies',# head count is not relevant to this analysis as we are focusing on rooms reserved instead
    'meal', # irrelevant to current analysis as we aren't examining meals
    'market_segment', # information is vague unless clarifying information is provided
    'distribution_channel', # information is vague unless clarifying information is provided
    'is_repeated_guest', # could be useful in viewing returning customer trends but not currently relevant
    'previous_cancellations', # unclear on this columns usage
    'previous_bookings_not_canceled', # unclear on this columns usage
    'booking_changes', # not relevant as we dont know exactly what constitutes a change and how it would affect this analysis
    'required_car_parking_spaces', # does not represent the customer base well as many customers do not require parking
    'total_of_special_requests', # not relevant to customer booking trends
    'days_in_waiting_list', # less than 500 out of roughly 120k show any wait at all. this information is statistically irrelevant
    'deposit_type' # doesn't tell us about anything we need to know for this analysis
]
for _ in drop_list:
    df_iqr.drop(_, axis=1, inplace=True)

df_iqr.head().reset_index() #visual check

,index,hotel,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,country,reserved_room_type,assigned_room_type,agent,company,customer_type,adr,reservation_status,reservation_status_date,check_in_date
0,2,Resort Hotel,0,7,0,1,GBR,A,C,0.0,0.0,Transient,75.0,Check-Out,2015-07-02,2015-07-01
1,3,Resort Hotel,0,13,0,1,GBR,A,A,304.0,0.0,Transient,75.0,Check-Out,2015-07-02,2015-07-01
2,4,Resort Hotel,0,14,0,2,GBR,A,A,240.0,0.0,Transient,98.0,Check-Out,2015-07-03,2015-07-01
3,5,Resort Hotel,0,14,0,2,GBR,A,A,240.0,0.0,Transient,98.0,Check-Out,2015-07-03,2015-07-01
4,7,Resort Hotel,0,9,0,2,PRT,C,C,303.0,0.0,Transient,103.0,Check-Out,2015-07-03,2015-07-01


In [8]:
# we just need the total nights count so I will combine these two and trim the extra columns
df_iqr['stays_in_nights'] = df_iqr['stays_in_weekend_nights'] + df_iqr['stays_in_week_nights']

# dropping columns
df_iqr.drop('stays_in_weekend_nights', axis=1, inplace=True)
df_iqr.drop('stays_in_week_nights', axis=1, inplace=True)

In [9]:
# factoring the checkout date by adding the stays in nights count to the check in date
df_iqr['check_out_date'] = df_iqr['check_in_date'] + pd.to_timedelta(df_iqr['stays_in_nights'], unit='D') # stays in nights needs to be made into timedelta so it can be added to check in date
# this makes stays in nights irrelevant... dropping below
df_iqr.drop('stays_in_nights', axis=1, inplace=True)
# this also makes the reservation status date irrelevant as we can gauge the last status by the checkout date for any record of relevance.
# in other words for any customer that checked out the check out date would also be the last reservation status update
df_iqr.drop('reservation_status_date', axis=1, inplace=True) # dropping reservation status date
df_iqr.head().reset_index()

,index,hotel,is_canceled,lead_time,country,reserved_room_type,assigned_room_type,agent,company,customer_type,adr,reservation_status,check_in_date,check_out_date
0,2,Resort Hotel,0,7,GBR,A,C,0.0,0.0,Transient,75.0,Check-Out,2015-07-01,2015-07-02
1,3,Resort Hotel,0,13,GBR,A,A,304.0,0.0,Transient,75.0,Check-Out,2015-07-01,2015-07-02
2,4,Resort Hotel,0,14,GBR,A,A,240.0,0.0,Transient,98.0,Check-Out,2015-07-01,2015-07-03
3,5,Resort Hotel,0,14,GBR,A,A,240.0,0.0,Transient,98.0,Check-Out,2015-07-01,2015-07-03
4,7,Resort Hotel,0,9,PRT,C,C,303.0,0.0,Transient,103.0,Check-Out,2015-07-01,2015-07-03


## Inconsistent Data

Check for inconsistent data and address any that arises. As always, use code comments to illustrate your thought process.

In [10]:
# the reserved room type and the assigned room type are inconsistent so I will keep the reserved room type as it is more representative of customer demand
# dropping assigned room type
df_iqr.drop('assigned_room_type', axis=1, inplace=True)

In [11]:
# converting two columns that were imported from the csv as the wrong datatype to integers
df_iqr['agent'] = df_iqr['agent'].astype(int)
df_iqr['company'] = df_iqr['company'].astype(int)

In [12]:
df_iqr.dtypes #final data types check


hotel                         object
is_canceled                    int64
lead_time                      int64
country                       object
reserved_room_type            object
agent                          int64
company                        int64
customer_type                 object
adr                          float64
reservation_status            object
check_in_date         datetime64[ns]
check_out_date        datetime64[ns]
dtype: object

In [13]:
df_iqr.to_csv('cleaned_hotel_bookings.csv', index=False)

## Summarize Your Results

Make note of your answers to the following questions.

1. Did you find all four types of dirty data in your dataset?
2. Did the process of cleaning your data give you new insights into your dataset?
3. Is there anything you would like to make note of when it comes to manipulating the data and making visualizations?

# Answers

1. There were outliers, nulls, and inconsistencies. Duplicates were ignored as it is reasonably possible that customers could share the same data points on any given booking.

2. Cleaning the data helped me think about how I could extrapolate and arrange new data based on breadcrumbs left in the original csv. i.e. extrapolating the check-in/check-out dates from partial date information and length of stay.

3. Show trend information over time to glean the most effective insights.